In [1]:
#Imports
import os
import pickle
import numpy as np
from copy import deepcopy

In [2]:
cwd = os.getcwd()

f = open(os.path.join(cwd, r'WordMappingDic.pickle'), 'rb')
word_dic = pickle.load(f) #Word to integer mapping
f.close()

f = open(os.path.join(cwd, r'TagMappingDic.pickle'), 'rb')
tag_dic = pickle.load(f) #Tag to integer mapping
f.close()

f = open(os.path.join(cwd, r'MatrixA.pickle'), 'rb')
a = pickle.load(f) #States Transition Matrix
f.close()

f = open(os.path.join(cwd, r'MatrixB.pickle'), 'rb')
b = pickle.load(f) #Observation Matrix
f.close()

In [3]:
new_tag_dic = deepcopy(tag_dic) #New Tag Dict without START and END tags
new_tag_dic.pop('START')
new_tag_dic.pop('END')

37

In [4]:
f = open(os.path.join(cwd, r'Training set_HMM.txt'), 'r') #Read Test File
g = open(os.path.join(cwd, r'Output.txt'), 'w') #Output File
    
for line in f.read().split('\n\n'):
    wtp = line.split('\n')

    words = []
    for wt in wtp:
        if wt != '':
            w,_ = wt.split('\t')
            w.strip()        
            words.append(w)
    
    if (len(words) < 1):
        continue

    for i in range(len(words)): #Replace words not in training set by UNK
        if words[i] not in word_dic:
            words[i] = 'UNK'
            
    ##Viterbi Algo
    viterbi = np.zeros((len(tag_dic), len(word_dic))) #DP Array
    backptr = np.chararray((len(tag_dic), len(word_dic)), itemsize=6, unicode=True) #Back pointers for back tracking
    backptr[:] = 'START'
        
    #INITIALISATION
    for s in new_tag_dic.keys():
        viterbi[tag_dic[s]][word_dic[words[0]]] = a[tag_dic['START']][tag_dic[s]] + b[word_dic[words[0]]][tag_dic[s]]
        backptr[tag_dic[s]][word_dic[words[0]]] = 'START'
#         print(viterbi[tag_dic[s]][word_dic[words[0]]])
#         print(backptr[tag_dic[s]][word_dic[words[0]]])
#         print(s)
        
    #RECURSION
    for t in range(1, len(words)):
        for s in new_tag_dic.keys():
            
            maxx = -float('inf') 
            maxx_item = -1
            for s_ in new_tag_dic.keys():
                tmp = viterbi[tag_dic[s_]][word_dic[words[t-1]]] + a[tag_dic[s_]][tag_dic[s]] + b[word_dic[words[t]]][tag_dic[s]]
#                 print(tmp, maxx, maxx_item)
                if (tmp > maxx):
                    maxx = tmp
                    maxx_item = s_
#                 print(tmp, maxx, maxx_item)
#                 print()
                
            viterbi[tag_dic[s]][word_dic[words[t]]] = maxx
            backptr[tag_dic[s]][word_dic[words[t]]] = maxx_item
        
    #TERMINATION
    maxx = -float('inf')
    maxx_item = -1
    T = len(words)-1
    
    for s in new_tag_dic.keys():
        tmp = viterbi[tag_dic[s]][word_dic[words[T]]]
        if (tmp > maxx):
            maxx = tmp
            maxx_item = s
    
    viterbi[tag_dic['END']][word_dic[words[T]]] = maxx
    backptr[tag_dic['END']][word_dic[words[T]]] = maxx_item
    
#     print(viterbi)
#     print(backptr)
    
    #Travesing the back pointers
    ts = backptr[tag_dic['END']][word_dic[words[T]]]
    ans = [ts]
    
    for word in reversed(words):
        ts = backptr[tag_dic[ts]][word_dic[word]]
        ans.append(ts)
        
    ans.reverse()
    
    for i in range(len(words)):
        g.write(words[i] + '\t' + ans[i+1] + '\n')
#         print(words[i], ans[i+1])
    g.write('\n')

g.close()
f.close()

In [7]:
# #Compute Accuracy

# f = open(os.path.join(cwd, r'Training set_HMM.txt'), 'r')
# g = open(os.path.join(cwd, r'Output.txt'), 'r')

# originalpairs = []
# mypairs = []

# for line in f:
#     if (line != '\n'):
#         w,t = line.strip().split('\t')
#         originalpairs.append((w,t)) 

# for line in g:
#     if (line != '\n'):
#         w,t = line.strip().split('\t')
#         mypairs.append((w,t)) 
    
# g.close()
# f.close()

In [8]:
# count = 0

# for i in range(len(mypairs)):
#     if (mypairs[i][1] == originalpairs[i][1]):
#         count += 1

# print(count/len(mypairs))

0.8957434052757793
